# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7fcabc86f130>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91420,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-06,NaN,NaN,NaN,NaN,...,NaN,69.44,NaN,73.81,NaN,55.56,NaN,60.26,NaN,25.0
91421,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-07,NaN,NaN,NaN,NaN,...,NaN,69.44,NaN,73.81,NaN,55.56,NaN,60.26,NaN,25.0
91422,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-08,NaN,NaN,NaN,NaN,...,NaN,69.44,NaN,73.81,NaN,55.56,NaN,60.26,NaN,25.0
91423,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-12-09,NaN,NaN,NaN,NaN,...,NaN,69.44,NaN,73.81,NaN,55.56,NaN,60.26,NaN,25.0


In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

<ipython-input-7-9010e734381b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)


In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

<ipython-input-8-a3c4998d92a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)


In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91288,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91289,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91290,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91291,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [11]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [12]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols                     # sceglie le colonne "NewCases" + tutti i NPIs
y_col = cases_col                                 # L'output è solo il numero di Nuovi Casi
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()

for g in geo_ids:
    
    gdf = df[df.GeoID == g]                   # Dataframe per il GeoID
    all_case_data = np.array(gdf[cases_col])  # nuovi casi in ordine cronologico shape = (len(gdf), 1)
    all_npi_data = np.array(gdf[npi_cols])    # npi in ordine cronologico shape = (len(gdf), 12)

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)  # questo è il numero totale di giorni che abbiamo a disposizione
    
    for d in range(nb_lookback_days, nb_total_days - 1): # questo for va da 30 alla fine
        X_cases = all_case_data[d - nb_lookback_days: d] # prende i range 0-30, 1-31, 2-32 etc...
        # X_cases è (30 x 1)
        
        
        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d] # Higher NPI means LOWER NewCases, stessi range sopra
        # X_npis è (30 x 12)

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(), X_npis.flatten()]) # 1 sample è (30 NC + 30 x 12 PIs,)
        # X_sample è un vettore con in fila tutti i NC e NPI, 
        # come se lo spazio fosse composto da 390 variabili
        
        y_sample = all_case_data[d + 1] # L'output è semplicemente il giorno dopo
        
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
# Scelta Randomica di test e train set.
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=42)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 146.2160023333569
Test MAE: 147.25539300003004


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Questo rientra nel contesto dell'explainability 

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.05050961552680664
Day -6 NewCases 0.4552500352887967
Day -5 NewCases 0.25749302407890673
Day -4 NewCases 0.022780152684931113
Day -3 NewCases 0.012340191672471638
Day -2 NewCases 0.0750628314353441
Day -1 NewCases 0.1991557707195756
Day -26 C6_Stay at home requirements 0.6963656395455212
Day -25 C6_Stay at home requirements 5.915953267092885
Day -21 C2_Workplace closing 14.432620626122691
Intercept 29.9927582188501


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [25]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [26]:
%%time
preds_df = predict_df("2020-02-01", "2020-09-30", path_to_ips_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv", verbose=False)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

...

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)
(390,)

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [52]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../covid_xprize/validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [53]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,142.56629738919912
Aruba,,2020-08-02,161.10804722795518
Aruba,,2020-08-03,174.4263128199457
Aruba,,2020-08-04,197.16095044264176
Afghanistan,,2020-08-01,266.70666571128135
Afghanistan,,2020-08-02,361.78323642335
Afghanistan,,2020-08-03,373.63396107727465
Afghanistan,,2020-08-04,379.39879996232133
Angola,,2020-08-01,317.3563234888792


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [23]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [34]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

validate(start_date="2020-02-01",
         end_date="2020-09-30",
         ip_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/try_1_month.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!
Generating predictions from 2020-02-01 to 2020-09-30...
Saved predictions to predictions/try_1_month.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [25]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../covid_xprize/validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 72.6 ms, sys: 9.12 ms, total: 81.7 ms
Wall time: 2.01 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [26]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-16
End date: 2021-06-14


In [27]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [28]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-16 to 2021-06-14...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 6.54 s, sys: 824 ms, total: 7.37 s
Wall time: 2min 35s
